In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [2]:
# Load and merge data
energy_df = pd.read_csv('Resources/Total_consumption_for_all_sectors.csv')

# Drop the first two rows and any rows with United States in the description regardless of what follows
energy_df = energy_df.drop([0, 1])
energy_df = energy_df[~energy_df['description'].str.contains('United States')].reset_index(drop=True)

# Drop source key column
energy_df = energy_df.drop(columns=['source key'])

# Change NaN values and '--' to 0
energy_df = energy_df.fillna(0)
energy_df = energy_df.replace('--', 0)

energy_df.head(10)


,description,units,1-Jan,1-Feb,1-Mar,1-Apr,1-May,1-Jun,1-Jul,1-Aug,...,23-Dec,24-Jan,24-Feb,24-Mar,24-Apr,24-May,24-Jun,24-Jul,24-Aug,24-Sep
0,New England,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,New England : coal,thousand tons,772,698,749,638,484,634,693,727,...,15,15,5,10,2,3,3,23,11,5
2,New England : petroleum liquids,thousand barrels,4928,3283,3520,1731,2346,2520,2170,3475,...,48,197,44,35,NM,52,92,99,68,31
3,New England : petroleum coke,thousand tons,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,New England : natural gas,thousand Mcf,15773,16570,21465,15375,21527,23526,26465,32238,...,37244,40476,31890,35342,28059,30274,39674,50246,47339,38520
5,Connecticut,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Connecticut : coal,thousand tons,138,149,160,128,131,143,146,148,...,0,0,0,0,0,0,0,0,0,0
7,Connecticut : petroleum liquids,thousand barrels,1525,828,1330,491,650,755,523,1061,...,7,71,10,10,NM,NM,21,NM,NM,11
8,Connecticut : petroleum coke,thousand tons,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Connecticut : natural gas,thousand Mcf,1423,1865,2869,1130,1727,2508,3095,4812,...,17570,15083,12948,14769,11531,12642,15991,19955,19086,16665


In [3]:
# Drop rows where description is only a state and not a state:sector
energy_df = energy_df[energy_df['description'].str.contains(':')].reset_index(drop=True)

# Split the description column into state and fuel_type columns and add them to the front of the dataframe
energy_df[['state', 'fuel_type']] = energy_df['description'].str.split(':', expand=True)
energy_df = energy_df[['state', 'fuel_type'] + [col for col in energy_df.columns if col not in ['state', 'fuel_type']]]
energy_df['state'] = energy_df['state'].str.strip()
energy_df['fuel_type'] = energy_df['fuel_type'].str.strip()

# Drop the description column
energy_df = energy_df.drop(columns=['description'])

energy_df.head()


,state,fuel_type,units,1-Jan,1-Feb,1-Mar,1-Apr,1-May,1-Jun,1-Jul,...,23-Dec,24-Jan,24-Feb,24-Mar,24-Apr,24-May,24-Jun,24-Jul,24-Aug,24-Sep
0,New England,coal,thousand tons,772,698,749,638,484,634,693,...,15,15,5,10,2,3,3,23,11,5
1,New England,petroleum liquids,thousand barrels,4928,3283,3520,1731,2346,2520,2170,...,48,197,44,35,NM,52,92,99,68,31
2,New England,petroleum coke,thousand tons,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,New England,natural gas,thousand Mcf,15773,16570,21465,15375,21527,23526,26465,...,37244,40476,31890,35342,28059,30274,39674,50246,47339,38520
4,Connecticut,coal,thousand tons,138,149,160,128,131,143,146,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Remove rows where the state is not a state
energy_df = energy_df[energy_df['state'].isin(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
                                               'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
                                               'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
                                               'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
                                               'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
                                               'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
                                               'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
                                               'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
                                               'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
                                               'West Virginia', 'Wisconsin', 'Wyoming'])].reset_index(drop=True)
energy_df.head()

,state,fuel_type,units,1-Jan,1-Feb,1-Mar,1-Apr,1-May,1-Jun,1-Jul,...,23-Dec,24-Jan,24-Feb,24-Mar,24-Apr,24-May,24-Jun,24-Jul,24-Aug,24-Sep
0,Connecticut,coal,thousand tons,138,149,160,128,131,143,146,...,0,0,0,0,0,0,0,0,0,0
1,Connecticut,petroleum liquids,thousand barrels,1525,828,1330,491,650,755,523,...,7,71,10,10,NM,NM,21,NM,NM,11
2,Connecticut,petroleum coke,thousand tons,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Connecticut,natural gas,thousand Mcf,1423,1865,2869,1130,1727,2508,3095,...,17570,15083,12948,14769,11531,12642,15991,19955,19086,16665
4,Maine,coal,thousand tons,37,32,37,29,33,32,34,...,5,3,5,6,2,3,3,3,3,5


In [5]:
# Change any values = to NM to 0
energy_df = energy_df.replace('NM', 0)
energy_df.head()

,state,fuel_type,units,1-Jan,1-Feb,1-Mar,1-Apr,1-May,1-Jun,1-Jul,...,23-Dec,24-Jan,24-Feb,24-Mar,24-Apr,24-May,24-Jun,24-Jul,24-Aug,24-Sep
0,Connecticut,coal,thousand tons,138,149,160,128,131,143,146,...,0,0,0,0,0,0,0,0,0,0
1,Connecticut,petroleum liquids,thousand barrels,1525,828,1330,491,650,755,523,...,7,71,10,10,0,0,21,0,0,11
2,Connecticut,petroleum coke,thousand tons,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Connecticut,natural gas,thousand Mcf,1423,1865,2869,1130,1727,2508,3095,...,17570,15083,12948,14769,11531,12642,15991,19955,19086,16665
4,Maine,coal,thousand tons,37,32,37,29,33,32,34,...,5,3,5,6,2,3,3,3,3,5


In [6]:
# Assuming your DataFrame is named energy_df
# 1. Melt the DataFrame
melted_df = pd.melt(energy_df, 
                    id_vars=['state', 'fuel_type', 'units'], 
                    var_name='date', 
                    value_name='energy_consumption')

# 2. Extract year and month from the 'date' column
melted_df['year'] = melted_df['date'].apply(lambda x: 2000 + int(x.split('-')[0]))
melted_df['month'] = melted_df['date'].apply(lambda x: x.split('-')[1])

# 3. Convert month abbreviations to full month names
month_map = {
    'Jan': 'January', 'Feb': 'February', 'Mar': 'March', 'Apr': 'April',
    'May': 'May', 'Jun': 'June', 'Jul': 'July', 'Aug': 'August',
    'Sep': 'September', 'Oct': 'October', 'Nov': 'November', 'Dec': 'December'
}
melted_df['month'] = melted_df['month'].map(month_map)

# 4. Reorder columns and drop the original 'date' column
final_df = melted_df[['state', 'fuel_type', 'units', 'year', 'month', 'energy_consumption']]

# 5. Display the first few rows of the transformed DataFrame
final_df.head()

,state,fuel_type,units,year,month,energy_consumption
0,Connecticut,coal,thousand tons,2001,January,138
1,Connecticut,petroleum liquids,thousand barrels,2001,January,1525
2,Connecticut,petroleum coke,thousand tons,2001,January,0
3,Connecticut,natural gas,thousand Mcf,2001,January,1423
4,Maine,coal,thousand tons,2001,January,37


In [14]:
# Convert energy_consumption to numeric
final_df['energy_consumption'] = pd.to_numeric(final_df['energy_consumption'])

final_df.dtypes

C:\Users\Owner\AppData\Local\Temp\ipykernel_29764\1043333107.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['energy_consumption'] = pd.to_numeric(final_df['energy_consumption'])


state                 object
fuel_type             object
units                 object
year                   int64
month                 object
energy_consumption     int64
dtype: object

In [7]:
weather_df = pd.read_csv('Resources/usweather_month_1981-2023.csv')


In [9]:
# Change the st_abb column to state
weather_df = weather_df.rename(columns={'st_abb': 'state'})

# Change the abbreviated state name to full state name
state_map = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado',
    'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana',
    'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota',
    'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
    'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
    'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon',
    'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota',
    'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington',
    'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}
weather_df['state'] = weather_df['state'].map(state_map)
weather_df.head()

,state,st_code,ym,tmin,tmax,tavg,ppt
0,Alabama,1,198101,-3.205,11.428,4.112,34.136
1,Arkansas,5,198101,-3.541,10.039,3.249,27.502
2,Arizona,4,198101,0.343,14.894,7.619,17.308
3,California,6,198101,2.264,13.607,7.936,113.911
4,Colorado,8,198101,-9.283,7.325,-0.979,7.307


In [10]:
# Drop the st_code column
weather_df = weather_df.drop(columns=['st_code'])

weather_df['year'] = weather_df['ym'].astype(str).str[:4]
weather_df['month'] = weather_df['ym'].astype(str).str[4:6]

# Convert month numbers to month names (optional)
month_map = {
    '01': 'January', '02': 'February', '03': 'March', '04': 'April',
    '05': 'May', '06': 'June', '07': 'July', '08': 'August',
    '09': 'September', '10': 'October', '11': 'November', '12': 'December'
}
weather_df['month'] = weather_df['month'].map(month_map)

# Drop the ym column
weather_df = weather_df.drop(columns=['ym'])
weather_df.head()

,state,tmin,tmax,tavg,ppt,year,month
0,Alabama,-3.205,11.428,4.112,34.136,1981,January
1,Arkansas,-3.541,10.039,3.249,27.502,1981,January
2,Arizona,0.343,14.894,7.619,17.308,1981,January
3,California,2.264,13.607,7.936,113.911,1981,January
4,Colorado,-9.283,7.325,-0.979,7.307,1981,January


In [15]:
# Change year to numeric
weather_df['year'] = pd.to_numeric(weather_df['year'])
weather_df.dtypes

state     object
tmin     float64
tmax     float64
tavg     float64
ppt      float64
year       int64
month     object
dtype: object

In [18]:
merge_test_df = pd.merge(final_df, weather_df, on=['state', 'month', 'year'])
merge_test_df.head()

,state,fuel_type,units,year,month,energy_consumption,tmin,tmax,tavg,ppt
0,Connecticut,coal,thousand tons,2001,January,138,-9.013,1.500,-3.756,60.590
1,Connecticut,petroleum liquids,thousand barrels,2001,January,1525,-9.013,1.500,-3.756,60.590
2,Connecticut,petroleum coke,thousand tons,2001,January,0,-9.013,1.500,-3.756,60.590
3,Connecticut,natural gas,thousand Mcf,2001,January,1423,-9.013,1.500,-3.756,60.590
4,Maine,coal,thousand tons,2001,January,37,-17.020,-4.209,-10.614,31.682


In [20]:
# Export the merged DataFrame to a CSV file
merge_test_df.to_csv('Resources/energy_weather_data.csv', index=False)